In [1]:
""" Calculation of dynamical properties using the Lanczos method: 
Spectral function A(k,w) of the fermionic tV model """

import pylab as pl
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg.eigen.arpack as arp

def c_operators(L):
    """ Returns the fermionic annihilation operators c"""
    sp = sparse.csr_matrix(np.array([[0.,1.],[0.,0.]]))
    sz = sparse.csr_matrix(np.array([[1.,0.],[0.,-1.]]))
    
    c_list = []
    for i_site in range(L):
        if i_site==0:
            X=sp
        else:            
            X=sz
        for j_site in range(1,L): 
            if i_site==j_site:
                X=sparse.kron(X,sp, 'csr')
            else:
                if j_site < i_site:
                    X=sparse.kron(X, sz, 'csr')
                else:
                    X=sparse.kron(X, sparse.csr_matrix(np.eye(2)), 'csr')
        
        c_list.append(X)
    return c_list

def gen_hamiltonian(c_list,L): 
    """" Generates the Hamiltonian """    
    H_t = sparse.csr_matrix((2**L,2**L))
    H_V = sparse.csr_matrix((2**L,2**L))   
    
    t = np.ones(L); t[-1] = -1
    for i in range(L):
        H_t = H_t + t[i]*(c_list[i].T*c_list[np.mod(i+1,L)] + (c_list[i].T*c_list[np.mod(i+1,L)]).T)
        H_V = H_V + c_list[i].T*c_list[i]*c_list[np.mod(i+1,L)].T*c_list[np.mod(i+1,L)]
        
    return H_t,H_V
    



In [2]:
if __name__ == "__main__": 
    # Set parameters here
    L = 4 
    V = 0
    
    # Get ground state
    c_list = c_operators(L)
    H_t, H_V = gen_hamiltonian(c_list,L)
    
    H = -H_t + V*H_V
    E0,v = arp.eigsh(H,k=1,which='SA')
    E0 = E0.item()
    
    v[np.abs(v)<1e-10] = 0.
    v1=c_list[0].T*v
    print(v1)
    print("\n")
    print(H*v1/np.linalg.norm(H*v1))
    print(E0)

[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.35355339]
 [0.        ]
 [0.5       ]
 [0.35355339]
 [0.        ]]


[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.11022302e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-5.00000000e-01]
 [ 0.00000000e+00]
 [-7.07106781e-01]
 [-5.00000000e-01]
 [ 0.00000000e+00]]
-2.8284271247461907


Normorlize v1 also, we can find v1 and H*v1 is the same

In [4]:
print(-v1/np.linalg.norm(v1))

[[-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.5       ]
 [-0.        ]
 [-0.70710678]
 [-0.5       ]
 [-0.        ]]


Compare v1 and H*v1 (after normalization), the result is 0:

In [10]:
print(-v1/np.linalg.norm(v1) - H*v1)

[[-0.00000000e+00]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [ 1.11022302e-16]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [-0.00000000e+00]
 [-2.22044605e-16]
 [-0.00000000e+00]
 [ 1.11022302e-16]
 [-1.11022302e-16]
 [-0.00000000e+00]]
